In [ ]:
from imutils import contours
import imutils
import cv2


def extract_image(thresh,mask,dim,output_path,digit_number,ROI_number):
    ROI = thresh[y:y+h, x:x+w]
    borderoutput = cv2.copyMakeBorder(ROI, 30, 30, 30, 30, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    resized = cv2.resize(borderoutput, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(output_path+'\\'+digit_number+'\\ROI_{}.png'.format(ROI_number), resized)



def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return [x, y, w, h]

def _intersect(a,b):
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[0]+a[2], b[0]+b[2]) - x
    h = min(a[1]+a[3], b[1]+b[3]) - y
    if h<0:                                              # in original code :  if w<0 or h<0:
        return False
    return True


def intersect_area(a, b):  # returns None if rectangles don't intersect
    #dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dx = min(a[0]+a[2], b[0]+b[2]) - max(a[0], b[0])
    #dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    dy = min(a[1]+a[3], b[1]+b[3]) - max(a[1], b[1])
    if (dx>=0) and (dy>=0):
        return True
    return False


def _group_rectangles(rec):
    """
    Uion intersecting rectangles.
    Args:
        rec - list of rectangles in form [x, y, w, h]
    Return:
        list of grouped ractangles 
    """
    tested = [False for i in range(len(rec))]
    final = []
    i = 0
    while i < len(rec):
        if not tested[i]:
            j = i+1
            while j < len(rec):
                if not tested[j] and intersect_area(rec[i], rec[j]):
                    rec[i] = union(rec[i], rec[j])
                    tested[j] = True
                    j = i
                j += 1
            final += [rec[i]]
        i += 1

    return final
    

In [ ]:
import cv2
import numpy as np
from imutils import contours
import os

input_path = r'C:/Users/A/datasets/school_label_dataset/digits/'
output_path = r'C:\Users\A\datasets\school_label_dataset\extract_digits'

ROI_number = 0
for img_ in os.listdir(input_path):
    print(img_)
    bounding_boxes = []
    digit_number = img_.split('_')[0]
    image = cv2.imread(input_path+'\\'+img_)
    mask = np.zeros(image.shape, dtype=np.uint8)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    (cnts, _) = contours.sort_contours(cnts, method="left-to-right")
        
    dim = (28, 28)

    for c in cnts:
        area = cv2.contourArea(c)
        if area > 250:
            bounding_boxes.append(cv2.boundingRect(c))
            
    boundingBoxes = _group_rectangles(bounding_boxes)
    
    
    for (x, y, w, h) in boundingBoxes:
        print(ROI_number)
        ROI_number +=1  
        try:
            
            extract_image(thresh,mask,dim,output_path,digit_number,ROI_number)
            
        except:
            continue